In [2]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch import nn
from torch.optim import Adam
from torchvision.transforms import  Compose, ToTensor, Resize, RandomHorizontalFlip, RandomRotation, ColorJitter, RandomResizedCrop, Lambda
from PIL import Image
import matplotlib.pyplot as plt
from torchsummary import summary
import torchmetrics
from lightning.pytorch.loggers import WandbLogger
import wandb
import random
from datetime import datetime
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

### MacOS

Set start method to fork and device to metal

In [2]:
import torch.multiprocessing as mp
mp.set_start_method("fork", force=True)  # Ensure fork is used on macOS

device = torch.device("cpu")

if torch.mps.is_available():
    #torch.mps.set_buffer_capacity(1)
    #torch.mps.set_reuse_buffers(True)
    #torch.mps.initialize()
    device = torch.device("mps")
    num_workers = 11

In [1]:
print("CUDA verfügbar:", torch.cuda.is_available())
print("CUDA-Version:", torch.version.cuda)
print("GPU-Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Keine GPU gefunden")

if torch.cuda.is_available():
    device = torch.device("cuda")
    num_workers = 2


NameError: name 'torch' is not defined

In [4]:
# Dataset Path
DATASET_PATHS = ["../Dataset/local_dataset_all/train"]
DATASET_PATHS_TEST = ["../Dataset/local_dataset_all/test"]
DATASET_PATHS_VALID = ["../Dataset/local_dataset_all/val"] # Not used yet
DATASET_PATH_SPLIT = "../Dataset/MIT_local_data" # This dataset is split between train, val and test
PRICES_FILE_PATH = "../Dataset/prices.txt"
IMAGE_SIZE = (200, 200)
BATCH_SIZE = 64
SEED = 42

In [5]:
from lightning.pytorch import seed_everything

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
seed_everything(SEED)

Seed set to 42


42

In [6]:
import helper_functions as hp
num_classes = hp.get_num_classes(PRICES_FILE_PATH)
print(num_classes)

class ImagePriceDataset(Dataset):
    def __init__(self, dataset_paths, transform=None):
        self.files = []
        for dataset_path in dataset_paths:
            if not os.path.exists(dataset_path):
                raise ValueError(f"Path {dataset_path} doesn't exist.")
            for f in os.listdir(dataset_path):
                if f.endswith(".jpg"):
                    self.files.append((f, dataset_path))
        
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_name, dataset_path = self.files[idx]
        image, label = hp.load_image_labels_classify(file_name, dataset_path, IMAGE_SIZE, num_classes)

        # Wenn das Bild im float32 Format vorliegt, skaliere es und wandle es in uint8 um
        if image.dtype == np.float32:
            image = (image * 255).astype(np.uint8)  # Skalieren auf [0, 255] und in uint8 umwandeln

        # Konvertiere das numpy-Array in ein PIL-Image
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)
    
# Transforms
transform = Compose([
    #Resize(IMAGE_SIZE),               # Skaliere die Bilder auf die gewünschte Größe
    Lambda(lambda x: hp.custom_augmentation(x)),
    RandomHorizontalFlip(p=0.5),       # Zufälliges horizontales Spiegeln mit 50% Wahrscheinlichkeit
    RandomRotation(degrees=180),        # Zufällige Rotation um bis zu ±15 Grad
    #RandomResizedCrop(IMAGE_SIZE, scale=(0.9, 1.0)),
    ColorJitter(brightness=0.04,        # Zufällige Anpassung der Helligkeit
                contrast=0.1,          # Kontrast
                saturation=0.2,        # Sättigung
                hue=0.07),              # Farbton
    ToTensor()                         # Konvertiere das Bild zu einem Tensor
])

transform_test = Compose([
    ToTensor()                         # Konvertiere das Bild zu einem Tensor
])


# Dataset
train_dataset = ImagePriceDataset(DATASET_PATHS, transform=transform)
test_dataset = ImagePriceDataset(DATASET_PATHS_TEST, transform_test)
# Train-Valid Split
if len(DATASET_PATHS_VALID) > 0:
    val_dataset = ImagePriceDataset(DATASET_PATHS_VALID, transform=transform)
else:    
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

if DATASET_PATH_SPLIT is not None:
    dataset = ImagePriceDataset([DATASET_PATH_SPLIT], transform=transform_test) # We use no augmentation for the split dataset
    train_size = int(0.7 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_split_dataset, val_split_dataset, test_split_dataset = random_split(dataset, [train_size, val_size, test_size])
    train_dataset = ConcatDataset([train_dataset, train_split_dataset]) 
    val_dataset = ConcatDataset([val_dataset, val_split_dataset])
    test_dataset = ConcatDataset([test_dataset, test_split_dataset])


# DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

print(f"Training set size: {len(train_loader.dataset)} samples")
print(f"Validation set size: {len(val_loader.dataset)} samples")
print(f"Test set size: {len(test_loader.dataset)} samples")


61
Training set size: 18364 samples
Validation set size: 2338 samples
Test set size: 4276 samples


## Tenserflow Lite

In [3]:
import helper_functions as hp

model = hp.TrainModel.load_from_checkpoint("../Important-checkpoints/train_Efficientnet-classifier_20Jan-23:18:57.ckpt")
model = model.model
model.eval()
# Assuming you have a PyTorch model instance `model`
# and a dummy input tensor that matches your model's input size
dummy_input = torch.randn(1, 3, 200, 200)  # Adjust dimensions as needed

# Export the model to ONNX
torch.onnx.export(model.to("cpu"), dummy_input, "quantization/good-model.onnx", verbose=False)


/home/torgeschwark/miniconda3/envs/MLubu/lib/python3.12/site-packages/pytorch_lightning/utilities/parsing.py:209: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [5]:
from onnx2tf import convert

# Pfad zum ONNX-Modell
onnx_model_path = "quantization/good-model.onnx"

# Ausgabeordner für das TensorFlow-Modell
output_folder = "quantization/good-tf_model"

# Konvertierung durchführen
convert(
    input_onnx_file_path=onnx_model_path,  # Pfad zum ONNX-Modell
    output_folder_path=output_folder,  # Ordner für das konvertierte TensorFlow-Modell
    copy_onnx_input_output_names_to_tflite=True,  # Optional: ONNX-Ein-/Ausgabebenennungen behalten
)



Model optimizing started ============================================================
Traceback (most recent call last):
  File "/home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/site-packages/onnx2tf/onnx2tf.py", line 652, in convert
    result = subprocess.check_output(
  File "/home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/subprocess.py", line 421, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
  File "/home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "/home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/subprocess.py", line 1863, in _execute_child
    raise child_exception_type(errno_num, err_msg, err_filename

2025-01-24 16:03:04.294113: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


INFO: tf_op_type: convolution_v2
INFO:  input.1.input: name: tf.math.multiply_8/Mul:0 shape: (1, 1, 1, 8) dtype: <dtype: 'float32'> 
INFO:  input.2.weights: shape: (1, 1, 8, 32) dtype: <dtype: 'float32'> 
INFO:  input.3.bias: shape: (32,) dtype: <dtype: 'float32'> 
INFO:  input.4.strides: val: [1, 1] 
INFO:  input.5.dilations: val: [1, 1] 
INFO:  input.6.padding: val: SAME 
INFO:  input.7.group: val: 1 
INFO:  output.1.output: name: tf.math.add_3/Add:0 shape: (1, 1, 1, 32) dtype: <dtype: 'float32'> 

INFO: 13 / 246
INFO: onnx_op_type: Sigmoid onnx_op_name: /features/features.1/features.1.0/block/block.1/scale_activation/Sigmoid
INFO:  input_name.1: /features/features.1/features.1.0/block/block.1/fc2/Conv_output_0 shape: [1, 32, 1, 1] dtype: float32
INFO:  output_name.1: /features/features.1/features.1.0/block/block.1/scale_activation/Sigmoid_output_0 shape: [1, 32, 1, 1] dtype: float32
INFO: tf_op_type: sigmoid
INFO:  input.1.x: name: tf.math.add_3/Add:0 shape: (1, 1, 1, 32) dtype: <dt

I0000 00:00:1737730997.103665  394288 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1737730997.103984  394288 single_machine.cc:361] Starting new session
W0000 00:00:1737730997.401559  394288 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1737730997.401599  394288 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-01-24 16:03:17.590498: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:3893] Estimated count of arithmetic ops: 1.023 G  ops, equivalently 0.511 G  MACs
I0000 00:00:1737730997.791385  394288 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1737730997.791505  394288 single_machine.cc:361] Starting new session


wget https://github.com/PINTO0309/onnx2tf/releases/download/1.16.31/flatc.tar.gz && tar -zxvf flatc.tar.gz && sudo chmod +x flatc && sudo mv flatc /usr/bin/
Float32 tflite output complete!


W0000 00:00:1737730998.052349  394288 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1737730998.052390  394288 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-01-24 16:03:18.243038: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:3893] Estimated count of arithmetic ops: 1.023 G  ops, equivalently 0.511 G  MACs


wget https://github.com/PINTO0309/onnx2tf/releases/download/1.16.31/flatc.tar.gz && tar -zxvf flatc.tar.gz && sudo chmod +x flatc && sudo mv flatc /usr/bin/
Float16 tflite output complete!


In [1]:
import tensorflow as tf
import onnx
from tf2onnx.tf_loader import from_onnx_graph

# Lade das ONNX-Modell
onnx_model = onnx.load("quantization/good-model.onnx")

# Konvertiere es in ein TensorFlow-Modell
with tf.Graph().as_default() as tf_graph:
    tf_rep = from_onnx_graph(onnx_model, input_names=None, output_names=None)

# Speichere das TensorFlow-Modell
tf.saved_model.save(tf_rep, "quantization/good-tf_model")

2025-01-24 15:59:47.695351: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 15:59:47.695945: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-24 15:59:47.698586: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-24 15:59:47.704437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737730787.713902  394288 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737730787.71

ModuleNotFoundError: No module named 'tf2onnx'

In [1]:
import tensorflow as tf
import numpy as np
from torchvision.transforms import ToTensor

# Load images from the folder
image_dataset = tf.keras.utils.image_dataset_from_directory(
    "../Dataset/local_dataset_all/val",  # Path to the folder
    labels=None,                  # No labels if it's just images
    image_size=(200, 200),        # Resize images to 224x224
    batch_size=1               # Load images as individual tensors
)

# Convert the dataset to a NumPy array
x_train = np.array([image.numpy() for image in image_dataset])

print("Shape of x_train:", x_train.shape)

def representative_dataset():
  np.random.shuffle(x_train)
  for sample in x_train[:50]:
    sample = sample.astype(np.float32) / 255.0
    print(sample)
    yield [sample.astype(np.float32)]

'''def representative_dataset():
  for data in tf.data.Dataset.from_tensor_slices((x_train)).batch(1).take(100):
    yield {"image": tf.dtypes.cast(data, tf.float32)}'''

converter = tf.lite.TFLiteConverter.from_saved_model("/home/torgeschwark/TinyML-MT/saved_model")

converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Default optimizations, which include quantization
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  
converter.inference_output_type = tf.int8  


tflite_model_quantized = converter.convert()

# Save the optimized model to a file
with open("quantization/good-model_quantized.tflite", "wb") as f:
    f.write(tflite_model_quantized)


2025-01-24 17:35:09.386257: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 17:35:09.476152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737736509.515429  680207 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737736509.527359  680207 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 17:35:09.601249: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Found 540 files.


I0000 00:00:1737736512.569605  680207 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5558 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6
W0000 00:00:1737736514.334014  680342 gpu_backend_lib.cc:579] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykern/cuda_nvcc
  
  /usr/local/cuda
  /home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  /home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/site-packages/tensorflow/python/platf

Shape of x_train: (540, 1, 200, 200, 3)


W0000 00:00:1737736516.242936  680207 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1737736516.242999  680207 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-01-24 17:35:16.243301: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /home/torgeschwark/TinyML-MT/saved_model
2025-01-24 17:35:16.248027: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-24 17:35:16.248053: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /home/torgeschwark/TinyML-MT/saved_model
I0000 00:00:1737736516.262258  680207 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-01-24 17:35:16.263270: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-24 17:35:16.284003: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /home/torgeschwark/TinyML-MT/saved_model
2025-01-24

[[[[0.15686275 0.07058824 0.12156863]
   [0.22941177 0.1254902  0.16470589]
   [0.28627452 0.14117648 0.18235295]
   ...
   [0.62352943 0.25882354 0.19411765]
   [0.6098039  0.24901961 0.19803922]
   [0.59607846 0.24705882 0.19215687]]

  [[0.09411765 0.0254902  0.06862745]
   [0.1254902  0.04705882 0.06862745]
   [0.18039216 0.04705882 0.06078431]
   ...
   [0.65294117 0.29215688 0.22156863]
   [0.5862745  0.23137255 0.17058824]
   [0.5764706  0.22745098 0.16470589]]

  [[0.21372549 0.04901961 0.07450981]
   [0.5254902  0.29607844 0.31176472]
   [0.80784315 0.52156866 0.5294118 ]
   ...
   [0.65294117 0.29607844 0.22352941]
   [0.6372549  0.28431374 0.21372549]
   [0.6019608  0.2529412  0.18235295]]

  ...

  [[0.7352941  0.40588236 0.36666667]
   [0.7490196  0.42352942 0.37843138]
   [0.7490196  0.42745098 0.38235295]
   ...
   [0.70980394 0.46078432 0.53333336]
   [0.7627451  0.527451   0.6509804 ]
   [0.7647059  0.5411765  0.6666667 ]]

  [[0.72745097 0.39803922 0.35490197]
   [0.7

fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
2025-01-24 17:35:24.981359: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:3893] Estimated count of arithmetic ops: 1.023 G  ops, equivalently 0.511 G  MACs


In [5]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os

DATA_PATH = "../Dataset/local_dataset_all/test"  # Pfad zu den Testdaten (Ordner mit Bildern)
NET_PATH = "./quantization/good-model_quantized.tflite"  # Pfad zum TFLite-Modell

# TFLite-Modell laden
interpreter = tf.lite.Interpreter(model_path=NET_PATH)
interpreter.allocate_tensors()

# Eingabe- und Ausgabetensoren abrufen
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Details:", input_details)
print("Output Details:", output_details)

input_scale, input_zero_point = input_details[0]['quantization']
print(input_scale, input_zero_point)
output_scale, output_zero_point = output_details[0]['quantization']
print(output_scale, output_zero_point)


# Beispiel: Falls dein Eingabebild als float32 vorliegt
def preprocess_input(input_data, input_scale, input_zero_point):
    input_data = np.round(input_data / input_scale + input_zero_point).astype(np.int8)
    return input_data

def dequantize_output(output_data, output_scale, output_zero_point):
    return (output_data.astype(np.float32) - output_zero_point) * output_scale

def load_and_preprocess_image(image_path, target_size):
    """Lädt ein Bild und verarbeitet es vor."""
    image = Image.open(image_path)  # JPG in RGB konvertieren
    image = image.resize(target_size)  # Größe anpassen
    image_array = np.asarray(image, dtype=np.float32) / 255 # In ein NumPy-Array umwandeln
    image_array = np.expand_dims(image_array, axis=0)  # Batch-Dimension hinzufügen

    return preprocess_input(image_array, input_scale, input_zero_point)

# Größe aus dem Modell holen
input_shape = input_details[0]['shape']
target_size = (input_shape[1], input_shape[2])  # Höhe und Breite
# Alle Bilder im Ordner laden

image_files = [os.path.join(DATA_PATH, f) for f in os.listdir(DATA_PATH) if f.endswith('.jpg')]

for image_path in image_files:
    # print(f"Verarbeite Bild: {image_path}")

    input_data = load_and_preprocess_image(image_path, target_size)
    # print("quantized Input:", input_data)

    # Eingabedaten setzen
    input_type = input_details[0]['dtype']
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Inferenz ausführen
    interpreter.invoke()
    
    # Ausgabe abrufen
    output_data = interpreter.get_tensor(output_details[0]['index'])
    output_type = output_details[0]['dtype']
    # print("Raw Output Data (int8):", output_data)
    
    output_data = dequantize_output(output_data, output_scale, output_zero_point)
    # Hier runden wir die Werte auf ganze Zahlen
    output_data = np.round(output_data).astype(int)  # Rundet auf die nächsten Ganzzahlen

    # print("quantized output", output_data)

    print(f"Vorhersage für {image_path}: {output_data}")



Input Details: [{'name': 'serving_default_input.1:0', 'index': 0, 'shape': array([  1, 200, 200,   3], dtype=int32), 'shape_signature': array([  1, 200, 200,   3], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.003921568859368563, -128), 'quantization_parameters': {'scales': array([0.00392157], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output Details: [{'name': 'PartitionedCall:0', 'index': 417, 'shape': array([ 1, 61], dtype=int32), 'shape_signature': array([ 1, 61], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.012841911055147648, -122), 'quantization_parameters': {'scales': array([0.01284191], dtype=float32), 'zero_points': array([-122], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
0.003921568859368563 -128
0.012841911055147648 -122
Vorhersage für ../Dataset/local_dataset_all/test/image_2355.jpg: [[0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
def test_tflite(model_path, x_test, y_test):
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    
    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Prepare the test data (ensure it's in the correct format)
    input_shape = input_details[0]['shape']
    input_dtype = input_details[0]['dtype']
    
    # Convert x_test to match the input tensor's expected format
    x_test = x_test.astype(input_dtype)
    
    # Initialize variables for accuracy and predictions
    correct_predictions = 0
    all_predictions = []
    
    # Iterate through the test data and make predictions
    for i in range(len(x_test)):
        input_data = np.expand_dims(x_test[i], axis=0)  # Add batch dimension
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        
        # Get prediction
        output_data = interpreter.get_tensor(output_details[0]['index'])
        prediction = np.argmax(output_data, axis=1)[0]
        true_label = np.argmax(y_test[i], axis=0)
        
        if prediction == true_label:
            correct_predictions += 1
        
        all_predictions.append(prediction)
    
    # Calculate test accuracy
    accuracy = correct_predictions / len(x_test)
    print("Test accuracy:", accuracy)
    
    # Generate confusion matrix
    cm = confusion_matrix(np.argmax(y_test, axis=1), all_predictions)
    
    # Normalize confusion matrix by the sum of each row
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Create DataFrame for the confusion matrix with labels
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    
    # Plot the heatmap of the confusion matrix
    plt.figure(figsize=(4, 4))
    ax = sns.heatmap(cm * 100, annot=True, fmt='.1f', cmap="Blues", cbar=False)
    ax.set_ylabel("True Class", fontdict={'fontweight': 'bold'})
    ax.set_xlabel("Predicted Class", fontdict={'fontweight': 'bold'})
    plt.show()